____
__Universidad de San Andrés__<br/>
__Visión Artificial__<br/>
__Trabajo Práctico 1: Panorámica__<br/>
__Martin Bianchi y Federico Gutman__
____

### Importamos las librerías necesarias

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

### Visualizamos las imágenes

In [ ]:
# cargo las imágenes (open cv las carga en bgr)
udesa_left = cv2.imread("img/udesa_0.jpg", flags=cv2.IMREAD_COLOR)
udesa_center = cv2.imread("img/udesa_1.jpg", flags=cv2.IMREAD_COLOR)
udesa_right = cv2.imread("img/udesa_2.jpg", flags=cv2.IMREAD_COLOR)

# convierto a rgb
udesa_left = cv2.cvtColor(udesa_left, cv2.COLOR_BGR2RGB)
udesa_center = cv2.cvtColor(udesa_center, cv2.COLOR_BGR2RGB)
udesa_right = cv2.cvtColor(udesa_right, cv2.COLOR_BGR2RGB)

# muestro las imágenes
fig, ax = plt.subplots(1, 3, figsize=(15, 10))
ax[0].imshow(udesa_left)
ax[0].axis('off')

ax[1].imshow(udesa_center)
ax[1].axis('off')

ax[2].imshow(udesa_right)
ax[2].axis('off')
plt.show()

### Seguimos el pipeline propuesto

#### Paso 1: Detección de características visuales

In [ ]:
# se pueden usar: sift, surf, akaze, orb
# pruebo con sift

sift = cv2.SIFT_create()

# detecto keypoints y computo descriptores
keypoints_left, descriptors_left = sift.detectAndCompute(udesa_left, None)
keypoints_center, descriptors_center = sift.detectAndCompute(udesa_center, None)
keypoints_right, descriptors_right = sift.detectAndCompute(udesa_right, None)

# dibujo keypoints
img_keypoints_left = cv2.drawKeypoints(udesa_left, keypoints_left, None, color=(0,255,0), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
img_keypoints_center = cv2.drawKeypoints(udesa_center, keypoints_center, None, color=(0,255,0), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
img_keypoints_right = cv2.drawKeypoints(udesa_right, keypoints_right, None, color=(0,255,0), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)   

# muestro las imágenes con keypoints
fig, ax = plt.subplots(1, 3, figsize=(15, 10))
ax[0].imshow(img_keypoints_left)
ax[0].axis('off')

ax[1].imshow(img_keypoints_center)
ax[1].axis('off')

ax[2].imshow(img_keypoints_right)
ax[2].axis('off')
plt.show()


#### Paso 2: Supresión de No Máxima Adaptativa

In [ ]:
def anms(keypoints, num_points):
    if len(keypoints) <= num_points:
        return keypoints

    # Ordenar keypoints por su respuesta (fuerza del corner)
    keypoints = sorted(keypoints, key=lambda x: x.response, reverse=True)

    # Inicializar radios de supresión
    radii = [float('inf')] * len(keypoints)

    # Calcular radios de supresión
    for i in range(len(keypoints)):
        for j in range(i):
            if keypoints[j].response > keypoints[i].response:
                dist = np.linalg.norm(np.array(keypoints[i].pt) - np.array(keypoints[j].pt))
                if dist < radii[i]:
                    radii[i] = dist

    # Asociar radios a keypoints y ordenar por radio
    keypoints_with_radii = list(zip(keypoints, radii))
    keypoints_with_radii.sort(key=lambda x: x[1], reverse=True)

    # Seleccionar los mejores keypoints basados en los radios
    selected_keypoints = [kp for kp, r in keypoints_with_radii[:num_points]]

    return selected_keypoints   

#### Paso 3: Descripción de las características